### 1. Function generate touch point

In [12]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import math
def touch_generate(xMin = 1,xMax  = 4, yMin = 1, yMax = 3, fNorm = 128, eSilence = 3,
                   radiusMin = 0.2, radiusMax = 1.0, F = np.random.uniform(60.0, 128.0) , kRatio1 = 20, kRatio2 = 3):
    cols = []
    rows = []
    for i in range(xMin, xMax+1):
        cols.append(i)
    for j in range(yMin, yMax+1):
        rows.append(j)
    silence = np.random.normal(fNorm, eSilence, size = [yMax-yMin + 1, xMax-xMin + 1])
    for i in range(0,len(silence)):
        for j in range(0,len(silence[i])):
            silence[i][j] = int(silence[i][j])
    df = pd.DataFrame(silence, index = rows, columns = cols)
    RADIUS = round(np.random.uniform(radiusMin, radiusMax),2)
    radius = math.ceil(RADIUS)
    x = round(np.random.uniform((xMax-xMin+1)/2,(xMax-xMin+1)/2+0.99),2)     #x-coord touch point (horizontal)
    y = round(np.random.uniform((yMax-yMin)/2+1,(yMax-yMin)/2+1.99),2)      #y-coord touch point (vertical)     
    X = int(x)      #x-coord touch point (horizontal)
    Y = int(y)      #y-coord touch point (vertical)     
    for j in range(Y-radius, Y+radius+1):
        for i in range(X-radius, X+1):
            if (abs(X-i)+abs(Y-j))<2*radius:
                
                if i<xMin:
                    i = xMin
                if j<yMin:
                    j = yMin
                if j>yMax:
                    j = yMax
                dist = np.sqrt(sum(pow(a-b, 2) for a, b in zip([i, j], [x, y])))
                df.loc[j,i] = int(fNorm + abs(np.random.normal(fNorm*math.exp(-dist),RADIUS)))
                if df.loc[j,i]<0:
                    df.loc[j,i] = 0
    for j in range(Y-radius, Y+radius+1):
        for i in range(X+1, X+radius+2):
            if (abs(X-i)+abs(Y-j))<2*radius+1:
                if i>xMax:
                    i = xMax
                if j<yMin:
                    j = yMin
                if j>yMax:
                    j = yMax
                dist = np.sqrt(sum(pow(a-b, 2) for a, b in zip([i, j], [x, y])))
                df.loc[j,i] = int(fNorm - abs(np.random.normal(fNorm*math.exp(-dist),RADIUS)))
                if df.loc[j,i]<0:
                    df.loc[j,i] = 0     
    pd.options.display.float_format = '{:,.0f}'.format
    df_numberTouch = df.copy()
    num = 1
    for i in rows:
        for j in cols:
            df_numberTouch.loc[i,j] = num
            num +=1
    numberTouch = df_numberTouch.loc[Y,X]
    return df, x, y, RADIUS
df, x, y, RADIUS = touch_generate()
print('x={}'.format(x))
print('y={}'.format(y))
print('radius={}'.format(RADIUS))
print('y={}'.format(y))
print('X={}'.format(int(x)))
print('Y={}'.format(int(y)))
cmap = sns.diverging_palette(220, 20, as_cmap=True)
df.style.format('{:.0f}').background_gradient(cmap=cmap, axis = None, high = 0.07)

x=2.05
y=2.61
radius=0.38
y=2.61
X=2
Y=2


,1,2,3,4
1,128,153,108,125
2,165,197,87,111
3,125,214,82,125


In [13]:
df_numpy = df.to_numpy()
ROWS = np.array(df_numpy[0],ndmin=1)   #values on rows
for i in range(1,len(df)):
    ROWS = np.append(ROWS,df_numpy[i],axis = 0)

### 2. Generate train and test datasets

In [15]:
length_train = 15000   #size train dataset
length_test = 5000    #size test dataset
df_train = pd.DataFrame(columns = ['value','xy'])
for i in range(0,length_train):
    df, x, y, RADIUS  = touch_generate()
    df_numpy = df.to_numpy()
    ROWS = np.array(df_numpy[0],ndmin=1)
    for i in range(1,len(df)):
        ROWS = np.append(ROWS,df_numpy[i],axis = 0)
    df_train = df_train.append({'value':ROWS, 'xy':np.array([x,y])},ignore_index=True)
df_test = pd.DataFrame(columns = ['value','xy'])
for i in range(0,length_test):
    df, x, y, RADIUS  = touch_generate()
    df_numpy = df.to_numpy()
    ROWS = np.array(df_numpy[0],ndmin=1)
    for i in range(1,len(df)):
        ROWS = np.append(ROWS,df_numpy[i],axis = 0)
    df_test = df_test.append({'value':ROWS,'xy':np.array([x,y])},ignore_index=True)

### 3. Save train and test datasets

In [16]:
with open('df_train.pickle','wb') as file:
    pickle.dump(df_train,file)
with open('df_test.pickle','wb') as file:
    pickle.dump(df_test,file)    